In [15]:
import os
import json
import pandas as pd
import traceback

In [28]:
from langchain.chat_models import ChatOpenAI

In [34]:
from dotenv import load_dotenv

load_dotenv()

True

In [35]:
KEY=os.getenv("OPENAP_API_KEY")

In [9]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo",temparture=0.5)

NameError: name 'ChatOpenAI' is not defined

In [4]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [1]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
    "4": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },       
}

In [2]:
TEMPLATE ="""
Text:{text}
you are an expert MCQ maker. Given the above text, it is your job to \
create a quiz of {number} multiple choice questions for{subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [6]:
quiz_generation_prompt =PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE   
)

In [8]:
print(quiz_generation_prompt)

input_variables=['number', 'response_json', 'subject', 'text', 'tone'] input_types={} partial_variables={} template='\nText:{text}\nyou are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for{subject} students in {tone} tone.\nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{response_json}\n'


In [10]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

NameError: name 'llm' is not defined

In [ ]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students/ \
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at par with the congnitive and analytical abilities of the studenes, \
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students ability.
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [ ]:
#shouldnt this be TEMPLATE2?
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"],template=TEMPLATE)

In [ ]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose =True)

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=["text","number","subject","tone","response_json"],
                         output_variables=["quiz","review"],verbose=True)

In [12]:

file_path =r'C:\dev\projs\mcqgen\data.txt'
with open(file_path,'r') as file:
    TEXT=file.read()

In [13]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [16]:
#serialise the python dictonary into a JSON-formated string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [ ]:
NUMBER =5
SUBJECT="Machine Learning"
TONE="simple"

In [ ]:
#https://pyton.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#how to setup token usage tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text" :TEXT,
            "number" : NUMBER,
            "subject" :SUBJECT,
            "tone" :TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(f"total tokens:{cb.total_tokens}")
print(f"prompt tokens:{cb.prompt_tokens}")
print(f"completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")


In [ ]:
quiz =response.get("quiz")

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz_table_data =[]
for key, value in quiz.items():
    mcq=value["mcq"]
    options =" | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
        ]
    )
    correct =value["correct"]
    quiz_table_data.append({"MCQ":mcq,"choices":options,"correct":correct})

In [ ]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("Machinelearning.csv",index=False)